In [1]:
import pandas as pd
bad = pd.read_csv('../data/bad/bad.csv')
normal=pd.read_csv('../data/neautral/neutral.csv')
hiniku_=pd.read_csv('../data/hiniku/hiniku_with_word.csv')

In [2]:
positivelist = list(hiniku_.dropna().positiveword.values)
hinikulist = list(hiniku_.dropna().hinikuword.values)
badwordlist = list(bad.word)#悪口単語をリストで抽出

In [3]:
'''
悪口ワードの含有
皮肉ワードの含有
ポジティブワードの含有

[形態素解析した時の単語数],[単純に悪口単語が文章に含まれているのかのフラグ], [形態素ごとに現れる悪口数]
[皮肉単語が含まれているかのフラグ],[形態素ごとに現れる皮肉数]
[ポジティブな単語が含まれているかのフラグ], [形態素ごとに現れるポジティブ単語数]

'''
import pandas as pd
# df = bad.copy()
df = pd.concat([hiniku_, normal])
df = pd.concat([bad, df])
import sys
sys.path.append('/Users/nagataeiki/.pyenv/versions/3.7.4/lib/python3.7/site-packages')
import numpy as np
from janome.tokenizer import Tokenizer
from tqdm import tqdm

tokenizer = Tokenizer()
#悪口が含まれている数のカウント, 単に含まれているのかのフラグを返す
def get_word_count(text, wordlist):
    textlist = []
    count=0
    flg=0
    for t in tokenizer.tokenize(text):
        if t.surface in wordlist:
            count+=1
    for w in wordlist:
        if w in text:
            flg=1
            return count, flg
    
    return count,flg


#dfと悪口辞書を入れて新しい特徴量を含んだdfを返す
def make_bad_feature(df, badwordlist, positivelist, hinikulist):
    col=["content","mophologics_num", "bad_per_mophologic", "simple_bad_flg","target", "hiniku_per_mophologic",\
         "simple_hiniku_flg", "positive_per_mophologic", "simple_positive_flg"]
    basedf = pd.DataFrame(columns=col)
    tokenizer = Tokenizer()
    for r in tqdm(df.iterrows()):
        text = str(r[1].content)
        target = r[1].type
        num = len(tokenizer.tokenize(text))
        badcount,badflg = get_word_count(text, badwordlist)
        hinikucount, hinikuflg = get_word_count(text, hinikulist)
        positivecount, positiveflg = get_word_count(text, positivelist)
        newdf = pd.DataFrame(np.array([text, num, badcount,badflg,target,\
                                       hinikucount, hinikuflg, positivecount,\
                                       positiveflg]).reshape(1,-1),columns=col)
        basedf = pd.concat([basedf, newdf])

    basedf["target"] = basedf.apply(lambda x: int(x["target"]), axis=1)
    return basedf
df =make_bad_feature(df, badwordlist, positivelist, hinikulist)
df.head(2)
print(df.shape)


750it [00:05, 131.96it/s]

(750, 9)


In [78]:
testdf = df.copy()
testdf.target = testdf.target.apply(lambda x: 0 if x==1 else 1)#x==1(悪口)の時0, その他は1
# testdf.target = testdf.target.apply(lambda x: 0 if x==2 else 1)#x==2(皮肉)の時0, その他は1
testdf.target.unique()
testdf.target.unique()
X = testdf.drop("target",axis=1).drop('content', axis=1)
y=testdf.target

In [85]:
'''
1対1で予測をする時
'''

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score
x_train, x_test, y_train, y_test = train_test_split(X.drop(['comment_id'],axis=1),y, test_size=0.2, random_state=42)
svm = SVC(C=1.0, random_state=42)
svm.fit(x_train, y_train)

pred_train = svm.predict(x_train)
pred_test = svm.predict(x_test)
print(f1_score(y_train, pred_train))
print(f1_score(y_test, pred_test))

0.7044569450454349
0.7260273972602739


,mophologics_num,bad_per_mophologic,simple_bad_flg,hiniku_per_mophologic,simple_hiniku_flg,positive_per_mophologic,simple_positive_flg,comment_id
0,1,1,1,0,0,0,0,0
1,1,1,1,0,0,0,0,1
2,6,2,1,0,0,0,0,2
3,2,0,1,0,0,0,0,3
4,3,0,1,0,0,0,0,4
...,...,...,...,...,...,...,...,...
2347,4,0,0,0,0,1,1,2347
2348,5,0,0,0,0,0,0,2348
2349,2,0,0,0,0,0,0,2349
2350,5,0,0,0,0,0,0,2350


In [27]:
"""
1対その他での学習
"""


from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


def vectorize(x_train,x_test, vec):
    x_train_vec = vec.fit_transform(x_train.content)
    x_test_vec = vec.transform(x_test.content)
    
    x_train_vec = pd.DataFrame(x_train_vec.todense())
    x_test_vec = pd.DataFrame(x_test_vec.todense())
    
    x_train_vec["comment_id"]=x_train.comment_id.values
    x_test_vec["comment_id"]=x_test.comment_id.values
    
    return x_train_vec,x_test_vec



def train_and_predict(df):

    X = df.drop("target",axis=1)
    y = df.target

    x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
    
    vec = TfidfVectorizer()
    x_train_vec, x_test_vec = vectorize(x_train, x_test, vec)
    train_content = pd.DataFrame(x_train.content.values, columns=["content"])
    test_content  = pd.DataFrame(x_test.content.values, columns=["content"])
    x_train.drop(["content"],axis=1,inplace=True)
    x_test.drop(["content"],axis=1,inplace=True)
    new_x_train = x_train.merge(x_train_vec, on="comment_id")#.drop('comment_id',axis=1)
    new_x_test  = x_test.merge(x_test_vec, on="comment_id")#.drop('comment_id',axis=1)
    new_x_train.drop('comment_id', axis=1, inplace=True)
    new_x_test.drop('comment_id', axis=1, inplace=True)
    x_train.drop(["comment_id"],axis=1,inplace=True)
    x_test.drop(["comment_id"],axis=1,inplace=True)

    svm = SVC(C=1.0, random_state=42)
    vec_model = OneVsRestClassifier(svm)
    vec_model.fit(new_x_train, y_train)
    model = OneVsRestClassifier(svm)
    model.fit(x_train, y_train)
    print('fin')

    return {
        "pred_train_with_vec":  vec_model.predict(new_x_train),
        "pred_test_with_vec": vec_model.predict(new_x_test),
        "pred_train":model.predict(x_train),
        "pred_test":model.predict(x_test),
        "x_train":x_train,
        "new_x_train":new_x_train,
        "y_train":y_train,
        "new_x_test":new_x_test,
        "x_test":x_test,
        "y_test":y_test,
        "train_content":train_content,
        "test_content":test_content,
#         "x_train_vec":x_train_vec,
#         "x_test_vec":x_test_vec
    }

In [5]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import numpy as np
'''
mxm,ここでは3x3のマトリックスを作成する
そこからf1値を計算する（同ディレクトリの画像参照）
'''

def create_3x3_matrix(true_y, pred_y):
    matrix = np.zeros(9)

    for t,p in zip(true_y, pred_y):
        if t==0:
            if p==0:matrix[0]+=1
            if p==1:matrix[1]+=1
            if p==2:matrix[2]+=1
        elif t==1:
            if p==0:matrix[3]+=1
            if p==1:matrix[4]+=1
            if p==2:matrix[5]+=1
        elif t==2:
            if p==0:matrix[6]+=1
            if p==1:matrix[7]+=1
            if p==2:matrix[8]+=1

    return matrix.reshape(3,3)
# matrix=create_3x3_matrix(y_train, pred_train)
# matrix

In [6]:
'''
使用例:
 calc_scores(matrix)
 
出力例:
{'recalls': array([0.31693989, 0.34710744, 0.28      ]),
 'Precisions': array([0.28292683, 0.41176471, 0.2565445 ]),
 'f1': 0.31587601668402826}
'''

def calc_scores(matrix):
    Recalls = np.zeros(3)
    for i in range(3):
        Recalls[i] = matrix[i][i]/sum(matrix[i])
    Precisions = np.zeros(3)
    pred_sum = matrix.sum(axis=0)
    for i, ps in enumerate(pred_sum):
        Precisions[i] = matrix[i][i]/ps
    f1_score = 2*Recalls.mean()*Precisions.mean()/(Recalls.mean()+Precisions.mean())
    return {
        "recalls":Recalls,
        "precisions":Precisions,
        "f1":f1_score
    }

In [7]:
# df1.to_csv('../data/clean/mount_data.csv', index=False)
# df1.to_excel('../data/clean/mount_data.xlsx', index=False)
# df1 = pd.read_excel('../data/clean/mount_data.xlsx')
# df1 = df1.dropna()
# df1["type"]=df1.target
# df1 = make_bad_feature(df1, badwordlist=badwordlist, positivelist=positivelist, hinikulist=hinikulist)

In [8]:
# df1[df1.target==0].shape[0], df1[df1.target==1].shape[0],df1[df1.target==2].shape[0]

In [9]:
'''
predのキー
dict_keys(['pred_train', 'pred_test', 'x_train', 'new_x_train', 'y_train',\
'new_x_test', 'x_test', 'y_test', 'train_content', 'test_content'])
'''

def print_miss_pred(pred, test=True, true_y=np.nan, pred_y=np.nan,df=np.nan):
    if test:
        true_y = pred["y_test"]
        pred_y = pred["pred_test"]
        df     = pred["test_content"]
    else:
        true_y = pred["y_train"]
        pred_y = pred["pred_train"]
        df     = pred["train_content"]


    idx_arr = []
    pred_judge=[]
    true_judge=[]
    for i in range(len(pred_y)):
        if pred_y[i]!=true_y.values[i]:
#             print(pred[i], true_y.values[i])
            idx_arr.append(i)
            pred_judge.append(pred_y[i])
            true_judge.append(true_y.values[i])

    def print_str_label(num):
        if num==0:return "ニュートラル"
        if num==1:return "悪口"
        if num==2:return "皮肉"
        return np.nan
    
    col=["content","true_label_id","true_label","pred_label_id","pred_label"]
    miss_data = pd.DataFrame(columns=col)
    for i,idx in enumerate(idx_arr):
        tmpdf = pd.DataFrame(np.array([df.iloc[idx].content, true_judge[i],\
                                          print_str_label(true_judge[i]), pred_judge[i],\
                                        print_str_label(pred_judge[i])]).reshape(1,-1),columns=col)
        miss_data = pd.concat([miss_data, tmpdf])
        
#             display(df.iloc[idx].content)
#             print('true label  *',true_judge[i] , (print_str_label(true_judge[i])))
#             print('pred label  *',pred_judge[i], (print_str_label(pred_judge[i])))
#             print("------------------------------------------------------------------------------------")
        
    print(miss_data.shape)
    return miss_data

In [10]:
import pandas as pd
df = pd.read_csv('../data/clean/mount_data.csv').dropna()
df["comment_id"] = df.index

In [11]:
'''
他のベクトル変換の例

x_train,x_test, y_train, y_test = train_test_split(content, target, random_state=42)

・バイナリ
vec = CountVectorizer(binary=True)
train_and_val(x_train, x_test, y_train, y_test, vec)

・カウント
print('count')
vec = CountVectorizer(binary=False)
train_and_val(x_train, x_test, y_train, y_test, vec)


・TfIdf
vec = TfidfVectorizer()
vectorize(x_train, x_test, y_train, y_test, vec)

・bigram
vec = TfidfVectorizer(ngram_range=(1,3))
train_and_val(x_train, x_test, y_train, y_test, vec)
'''

"\n他のベクトル変換の例\n\nx_train,x_test, y_train, y_test = train_test_split(content, target, random_state=42)\n\n・バイナリ\nvec = CountVectorizer(binary=True)\ntrain_and_val(x_train, x_test, y_train, y_test, vec)\n\n・カウント\nprint('count')\nvec = CountVectorizer(binary=False)\ntrain_and_val(x_train, x_test, y_train, y_test, vec)\n\n\n・TfIdf\nvec = TfidfVectorizer()\nvectorize(x_train, x_test, y_train, y_test, vec)\n\n・bigram\nvec = TfidfVectorizer(ngram_range=(1,3))\ntrain_and_val(x_train, x_test, y_train, y_test, vec)\n"

In [12]:
df.head(2)

,content,mophologics_num,bad_per_mophologic,simple_bad_flg,target,hiniku_per_mophologic,simple_hiniku_flg,positive_per_mophologic,simple_positive_flg,comment_id
0,ガイジ,1,1,1,1,0,0,0,0,0
1,くどい,1,1,1,1,0,0,0,0,1


In [ ]:
pred = train_and_predict(df)

In [31]:
print("no vectorize train")
matrix=create_3x3_matrix(pred["y_train"], pred["pred_train"])
print(matrix)
print(calc_scores(matrix))
print()
print()
print('no vectoizer test')
matrix=create_3x3_matrix(pred["y_test"], pred["pred_test"])
print(matrix)
print(calc_scores(matrix))
print()
print()

print("vectorize train")
matrix=create_3x3_matrix(pred["y_train"], pred["pred_train_with_vec"])
print(matrix)
print(calc_scores(matrix))
print()
print()
print('vectorize test')
matrix=create_3x3_matrix(pred["y_test"], pred["pred_test_with_vec"])
print(matrix)
print(calc_scores(matrix))

no vectorize train
[[517.  22.  53.]
 [252. 448.  46.]
 [191.  49. 134.]]
{'recalls': array([0.87331081, 0.60053619, 0.35828877]), 'precisions': array([0.53854167, 0.86319846, 0.5751073 ]), 'f1': 0.6339142135942205}


no vectoizer test
[[129.  12.  14.]
 [ 64. 106.  16.]
 [ 47.  11.  29.]]
{'recalls': array([0.83225806, 0.56989247, 0.33333333]), 'precisions': array([0.5375    , 0.82170543, 0.49152542]), 'f1': 0.5970851886381953}


vectorize train
[[527.  19.  46.]
 [246. 482.  18.]
 [187.  38. 149.]]
{'recalls': array([0.8902027 , 0.6461126 , 0.39839572]), 'precisions': array([0.54895833, 0.89424861, 0.69953052]), 'f1': 0.6778058715586218}


vectorize test
[[130.   9.  16.]
 [ 65. 104.  17.]
 [ 45.  11.  31.]]
{'recalls': array([0.83870968, 0.55913978, 0.35632184]), 'precisions': array([0.54166667, 0.83870968, 0.484375  ]), 'f1': 0.6025906257243245}


In [15]:
pred["new_x_train"]

,mophologics_num,bad_per_mophologic,simple_bad_flg,hiniku_per_mophologic,simple_hiniku_flg,positive_per_mophologic,simple_positive_flg,comment_id,0,1,...,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902
0,11,0,0,0,0,0,0,1003,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0,0,1,1,0,0,554,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,0,0,0,0,0,0,2334,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,0,1,0,0,0,0,192,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,0,0,0,0,0,679,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,2,0,0,0,0,0,0,1850,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1708,5,0,0,0,1,0,1,1132,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1709,17,0,0,1,1,0,1,1167,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1710,10,0,0,1,1,0,0,1331,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# print_miss_pred(pred, test=True, )


(6, 5)


,content,true_label_id,true_label,pred_label_id,pred_label
0,このばばぁ 頭おかしいな,1,悪口,2,皮肉
0,マジでナチス以下だな。内政すらうまくない,1,悪口,2,皮肉
0,保守はきれいな人多いな,2,皮肉,0,ニュートラル
0,今は紳士的,2,皮肉,0,ニュートラル
0,動物の習性って思って諦めるしかないな。,2,皮肉,0,ニュートラル
0,言葉だけは立派ですね,2,皮肉,0,ニュートラル


In [86]:
hiniku_list

NameError: name 'hiniku_list' is not defined

['の方が',
 'みたいなのが',
 'いい感じ',
 'ですしね',
 '無い',
 'かな？',
 '大丈夫か？',
 '何なの？',
 '大丈夫？',
 '意外と',
 'だけ',
 '出来な',
 'したら',
 '迷惑',
 'ないの？',
 '元気のいい',
 'だもの',
 'だけは',
 '言おうね',
 'わけない',
 '流石に',
 'って',
 'という',
 'だろ',
 'ですか',
 'けど',
 '何より',
 'だと',
 '流石に',
 '真面目',
 'たんだね',
 '＾＾',
 'ですよ',
 '空気読め',
 'だけど',
 'だな',
 'してる',
 'あわよくば',
 'なりましたね',
 '単純で',
 'ならない',
 'けど',
 'けど',
 'ばっか',
 'だなぁ',
 'のに',
 'わけがない',
 'しすぎ',
 '病院',
 'だけは',
 'だけは',
 'だよ？',
 '幼稚',
 '方が',
 '真面目',
 'すぎる',
 'ある意味',
 'だろな',
 'かな？',
 'けどね',
 'けど',
 'のに',
 'レベル',
 '個性的',
 'だけは',
 'じゃ',
 'ない',
 'しか',
 'だけ',
 '自称',
 'なんて',
 'だけど',
 'だな',
 'かな？',
 'ねーだろ']